# Algoritmo KNN
## Por: Matheus Mendonca Lopes, Otávio, Raphael Griffoni, Vinícius Leôncio

Base de dados usada: https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csvm